In [8]:
import pandas as pd
import numpy as np
from gensim import models 
import gensim
from hazm import *

In [19]:
digi_data = pd.read_excel("digi02.xlsx")

In [20]:
digi_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   product_id           100000 non-null  int64 
 1   product_title        100000 non-null  object
 2   title_en             100000 non-null  object
 3   user_id              100000 non-null  int64 
 4   likes                100000 non-null  int64 
 5   dislikes             100000 non-null  int64 
 6   verification_status  100000 non-null  object
 7   recommend            100000 non-null  object
 8   title                96730 non-null   object
 9   comment              99883 non-null   object
 10  advantages           42241 non-null   object
 11  disadvantages        34121 non-null   object
dtypes: int64(4), object(8)
memory usage: 9.2+ MB


In [21]:
digi_data.columns

Index(['product_id', 'product_title', 'title_en', 'user_id', 'likes',
       'dislikes', 'verification_status', 'recommend', 'title', 'comment',
       'advantages', 'disadvantages'],
      dtype='object')

In [22]:
digi_data_filtered = digi_data[["product_title","title_en","likes","dislikes",
                                "verification_status", "recommend","title","comment","advantages","disadvantages"]]

In [23]:
digi_data_filtered.head()

,product_title,title_en,likes,dislikes,verification_status,recommend,title,comment,advantages,disadvantages
0,ماوس بی‌سیم لاجیتک مدل M325,IT,0,0,verified,\N,NaN,واقعا عالیه. من که ازش خیلی راضیم,NaN,NaN
1,شارژر همراه شیاومی مدل NDY-02-AN با ظرفیت 1000...,AC,4,1,verified,recommended,واقعاً عالیه,سلام، قبل اینکه نظرم رو بگم میخواستم به یک موض...,"[""عمر طولانی\r"",""افت بسیار کم میزان شارژ\r"",""ا...","[""ندارد""]"
2,یدک پولیشر میکروفایبر مهسان مدل 20119,HW,1,0,verified,not_recommended,خیلی سخت حوله اش در میاد,گیره های فلزی خیلی سخت تا میشوند و لذا حوله را...,NaN,NaN
3,گوشی موبایل هوآوی آنر مدل 5X KIW-L21 دو سیم‌کارت,MO,6,11,verified,no_idea,گوشی مقرون به صرفه,همه چیز در رابطه با ظاهر این گوشی بسیار خوب اس...,"[""صفحه نمایش پرنور و با کیفیت\r"",""کیفیت بالای ...","[""کیفیت پایین اسپیکر\r"",""حاشیه خالی زیر صفحه ن..."
4,شارژر همراه شیائومی مدل Mi ظرفیت 5000 میلی آمپ...,AC,19,4,verified,no_idea,ابعاد، استحکام و نگهداری شارژ مناسب,اگر ظرفیتش براتون کافیه حتما بخرید._x000D_\nیه...,"[""ظریف و زیبا و باریک\r"",""بدنه محکم و با دوام""]","[""ظرفیت متوسط""]"


In [25]:
digi_data_filtered["title_en"].value_counts()

title_en
AC    16376
PC    12443
HW    10483
FA    10465
TC     9846
TS     7915
IT     6999
MA     6395
PA     5751
HA     4207
AV     3865
MO     2254
DF     1348
HC     1063
BC      524
FF       60
GF        4
GC        2
Name: count, dtype: int64

In [26]:
len(digi_data_filtered)

100000

In [31]:
No_dup_df = digi_data_filtered.drop_duplicates()

In [32]:
len(No_dup_df)

99996

In [35]:
No_dup_na = No_dup_df.dropna()

In [36]:
len(No_dup_na)

29250

In [37]:
No_dup_na.columns

Index(['product_title', 'title_en', 'likes', 'dislikes', 'verification_status',
       'recommend', 'title', 'comment', 'advantages', 'disadvantages'],
      dtype='object')

In [38]:
# def dummies(x,df):
#     temp = pd.get_dummies(df[x], drop_first = True)
#     df = pd.concat([df, temp], axis = 1)
#     df.drop([x], axis = 1, inplace = True)
#     return df
# df_encoded = dummies('Embarked',No_dup_na)

In [40]:
No_dup_na["recommend"].value_counts()

recommend
recommended        11377
\N                 10432
not_recommended     4234
no_idea             3207
Name: count, dtype: int64

In [41]:
filtered_No_dup_na = No_dup_na[No_dup_na["recommend"] != r"\N"]

In [42]:
filtered_No_dup_na.head()

,product_title,title_en,likes,dislikes,verification_status,recommend,title,comment,advantages,disadvantages
1,شارژر همراه شیاومی مدل NDY-02-AN با ظرفیت 1000...,AC,4,1,verified,recommended,واقعاً عالیه,سلام، قبل اینکه نظرم رو بگم میخواستم به یک موض...,"[""عمر طولانی\r"",""افت بسیار کم میزان شارژ\r"",""ا...","[""ندارد""]"
3,گوشی موبایل هوآوی آنر مدل 5X KIW-L21 دو سیم‌کارت,MO,6,11,verified,no_idea,گوشی مقرون به صرفه,همه چیز در رابطه با ظاهر این گوشی بسیار خوب اس...,"[""صفحه نمایش پرنور و با کیفیت\r"",""کیفیت بالای ...","[""کیفیت پایین اسپیکر\r"",""حاشیه خالی زیر صفحه ن..."
4,شارژر همراه شیائومی مدل Mi ظرفیت 5000 میلی آمپ...,AC,19,4,verified,no_idea,ابعاد، استحکام و نگهداری شارژ مناسب,اگر ظرفیتش براتون کافیه حتما بخرید._x000D_\nیه...,"[""ظریف و زیبا و باریک\r"",""بدنه محکم و با دوام""]","[""ظرفیت متوسط""]"
5,لپ تاپ 15 اینچی ایسوس مدل N501VW - A,IT,6,1,verified,recommended,اقرار بیش از حد در ایراد گرفتن,سلام دوستان،،_x000D_\nمنم مثه بعضی از دوستان ق...,"[""همه چیز که تو مشخصات اومده بجز 3 مورد کوچیک""]","[""تاچ پدش میتونست بهتر باشه از نظر سرعت پاسخ گ..."
6,اسپیکر بلوتوثی دیووم مدل Bluetune bean,IT,3,0,verified,recommended,بسیار خوب,من چند سالی هست که این اسپیکرو خریدم و واقعا ح...,"[""کیفیت ساخت بسیار خوب\r"",""کیفیت صدا بسیار عال...","[""ندارد""]"


In [43]:
filtered_No_dup_na["recommend"].value_counts()

recommend
recommended        11377
not_recommended     4234
no_idea             3207
Name: count, dtype: int64

In [45]:
Final_digi_kala_df_4_sentiment_analysis = filtered_No_dup_na.loc[:,["comment","recommend"]]

In [46]:
Final_digi_kala_df_4_sentiment_analysis

,comment,recommend
1,سلام، قبل اینکه نظرم رو بگم میخواستم به یک موض...,recommended
3,همه چیز در رابطه با ظاهر این گوشی بسیار خوب اس...,no_idea
4,اگر ظرفیتش براتون کافیه حتما بخرید._x000D_\nیه...,no_idea
5,سلام دوستان،،_x000D_\nمنم مثه بعضی از دوستان ق...,recommended
6,من چند سالی هست که این اسپیکرو خریدم و واقعا ح...,recommended
...,...,...
99968,بعداز یک ماه زرد میشه از هم در میره,not_recommended
99970,خیلی بوی بدی نمیده که بقیه میگن فقط تنها ایراد...,recommended
99983,گول ۱۰۰۰۰میلی آمپر رو نخورید خیلی پایین هست ظر...,not_recommended
99995,بسیار تمیز و زیبا...من که خیلی راضیم,recommended


In [47]:
Final_digi_kala_df_4_sentiment_analysis.to_csv("sentiment_digikala_data.csv",index=False, encoding="utf-8")

# End of extract data needed